In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
match_data = pd.read_csv('./match_data.csv', index_col = 0)
match_data.head()

,Date,Tm,Opp,Result,MP,FG,FGA,FG%,2P,2PA,...,2P.1,2PA.1,2P%.1,3P.1,3PA.1,3P%.1,FT.1,FTA.1,FT%.1,PTS.1
0,2021/3/27,LAC,PHI,W 122-112,240,45,85,0.529,34,59,...,29,51,0.569,10,26,0.385,24,33,0.727,112
1,2021/3/27,PHI,LAC,L 112-122,240,39,77,0.506,29,51,...,34,59,0.576,11,26,0.423,21,27,0.778,122
2,2021/3/27,MIL,NYK,L 96-102,240,36,87,0.414,22,51,...,24,49,0.490,16,44,0.364,6,8,0.750,102
3,2021/3/27,NYK,MIL,W 102-96,240,40,93,0.430,24,49,...,22,51,0.431,14,36,0.389,10,16,0.625,96
4,2021/3/27,HOU,MIN,W 129-107,240,50,92,0.543,32,53,...,23,45,0.511,13,40,0.325,22,28,0.786,107


In [4]:
match_data.shape

(2600, 31)

In [5]:
# change date from dd/mm/yyy to yyymmdd
match_data['Date_ori'] = match_data['Date'] 
match_data['Date'] = pd.to_datetime(match_data['Date'], infer_datetime_format=True)
match_data['Date'] = match_data['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
match_data['Date'] = match_data['Date'].apply(lambda x: x[0:4] + x[5:7] + x[8:10])

In [6]:
# get player data for a team in a certain match from the website
# return a new dataframe with one column (columns names: {FG_players, FGA_players, FG%_players, 3P_players...})

def get_player_data(tables, pos, OT = 0):
    if pos == 'first':
        t1, t2 = 0, 7 + OT
    else: # pos = 'second'
        t1, t2 = 8 + OT, 15 + OT*2

    # get the main table
    df_tm = tables[t1]
    df_tm.columns = ['Starters', 'MP', 'FG', 'FGA', 'FG%', '3P','3PA', '3P%', 'FT', 'FTA', 
                        'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-']
    df_tm = df_tm[df_tm['Starters'] != 'Reserves']
    
    # get the additional table   
    df_tm_add = tables[t2]
    df_tm_add.columns = ['Starters', 'MP', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%','DRB%', 'TRB%', 
                            'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
    df_tm_add = df_tm_add[df_tm_add['Starters'] != 'Reserves']
    
    # combine the main and the additional table as the complete player_df
    player_df = df_tm.merge(df_tm_add, on = ['Starters', 'MP'])
    
    # calculate the player performance by the player_df 
    def calculate_players_average(player_df):
        player_df = player_df.fillna(0)
        player_df = player_df.query('MP not in ["Did Not Play", "Did Not Dress", "Not With Team"] \
                                    and Starters not in ["Team Totals"]')

        def transform_mp(mp):
            minutes, seconds = mp.split(':')
            return (int(minutes) * 60 + int(seconds)) / (240 * 60)
        player_df['MP'] = player_df['MP'].apply(transform_mp)

        def weighted_column(column, df):
            df[column] = df[column].apply(float)
            result = df[column] * df['MP']
            return sum(result)

        lst = []
        for column in player_df.columns[2:]:
            value = weighted_column(column, player_df)
            lst.append([float(value)]) 
        average_df = pd.DataFrame(lst).T
        average_df.columns = [i + '_players'for i in player_df.columns[2:]]
        
        return average_df
    
    return calculate_players_average(player_df)


In [7]:
# get 2 possible urls for a match from the match dataset
def get_url(row):
    match_id = row['Date']+ '0'
    id1 = match_id + row['Tm'] 
    id2 = match_id + row['Opp'] 
    url1 = 'https://www.basketball-reference.com/boxscores/' + id1 + '.html'
    url2 = 'https://www.basketball-reference.com/boxscores/' + id2 + '.html'
    return url1, url2

In [8]:
# read the table from url
def get_table(url1, url2):
    try:
        tables = pd.read_html(url1)
        url = url1
    except:
        tables = pd.read_html(url2)
        url = url2
    return url, tables

In [9]:
# create players dataframe to save the players performance for each team by match sequence

player_df_columns = ['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       '+/-', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
players_df = pd.DataFrame(columns =  [i + '_players'for i in player_df_columns])

i = 0

while i < match_data.shape[0]:
    if i % 10 == 0:
        print ("Starting ", i)
    
    # read in every match data
    row1, row2 = match_data.iloc[i], match_data.iloc[i+1]
    tm1, tm2 = row1['Tm'], row2['Tm']
    
    # determine which team is the host team
    url1, url2 = get_url(row1)
    
    url, tables = get_table(url1, url2)
    host_team = url[-8:-5]
    
    # check if ot
    ot = 0
    if '(2OT)' in row1['Result']:
        ot = 2
    elif '(OT)' in row1['Result']:
        ot = 1
        
    # determine which team is described in the first/second table on the website
    df_team1 = get_player_data(tables, 'first', ot)
    df_team2 = get_player_data(tables, 'second', ot)
    
    df1 = df_team2 if host_team == tm1 else df_team1
    df2 = df_team1 if host_team == tm1 else df_team2
    
    # combine the data 
    players_df = pd.concat([players_df, df1, df2], axis = 0)      
    i = i + 2

Starting  0
Starting  10
Starting  20
Starting  30
Starting  40
Starting  50
Starting  60
Starting  70
Starting  80
Starting  90
Starting  100
Starting  110
Starting  120
Starting  130
Starting  140
Starting  150
Starting  160
Starting  170
Starting  180
Starting  190
Starting  200
Starting  210
Starting  220
Starting  230
Starting  240
Starting  250
Starting  260
Starting  270
Starting  280
Starting  290
Starting  300
Starting  310
Starting  320
Starting  330
Starting  340
Starting  350
Starting  360
Starting  370
Starting  380
Starting  390
Starting  400
Starting  410
Starting  420
Starting  430
Starting  440
Starting  450
Starting  460
Starting  470
Starting  480
Starting  490
Starting  500
Starting  510
Starting  520
Starting  530
Starting  540
Starting  550
Starting  560
Starting  570
Starting  580
Starting  590
Starting  600
Starting  610
Starting  620
Starting  630
Starting  640
Starting  650
Starting  660
Starting  670
Starting  680
Starting  690
Starting  700
Starting  710
Sta

In [10]:
# prepare players_df, match_data 
players_df = players_df.reset_index().drop(columns = ['index'])
match_data = match_data.drop(columns = ['Date']).rename(columns = {'Date_ori': 'Date'})
date = match_data.pop('Date')
match_data.insert(0, 'Date', date)

# combine the players data with the matches by index
df_all = pd.concat([match_data, players_df],  axis = 1)
df_all.head(2)

,Date,Tm,Opp,Result,MP,FG,FGA,FG%,2P,2PA,...,DRB%_players,TRB%_players,AST%_players,STL%_players,BLK%_players,TOV%_players,USG%_players,ORtg_players,DRtg_players,BPM_players
0,2021/3/27,LAC,PHI,W 122-112,240,45,85,0.529,34,59,...,15.669340,10.003326,14.443854,1.839944,1.174243,12.329424,20.008688,127.632847,114.770694,2.116778
1,2021/3/27,PHI,LAC,L 112-122,240,39,77,0.506,29,51,...,15.362375,10.012007,12.532625,1.027785,0.679583,9.768000,20.000146,115.643889,124.967014,-0.263049


In [11]:
df_all.shape

(2600, 65)

In [12]:
df_all.to_csv('./match_player_data.csv', index = 0)